# Mean-Variance Portfolio Optimization in Finance

## Motivation and Background

In quantitative finance, portfolio optimization is a fundamental problem: how should an investor allocate wealth among different assets to achieve the best trade-off between expected return and risk? A real-world scenario is an investment fund choosing weights for stocks in a portfolio. The classic approach, introduced by Harry Markowitz, is Mean-Variance Optimization (MVO) ￼

- Here, return is measured by the expected portfolio return (mean of outcomes) and risk is measured by the variance (or standard deviation) of portfolio returns. Markowitz’s Modern Portfolio Theory says an investor should maximize return for a given risk level or equivalently minimize risk for a desired return ￼. The set of optimal portfolios for all possible trade-offs forms the efficient frontier, which is the curve of minimal risk for each level of return (any portfolio not on this frontier is suboptimal in either risk or return) ￼.

## Real-world application: 

This optimization is used by asset managers and robo-advisors to allocate capital. For example, consider using historical data of $m=10$ stocks (say Apple, Google, etc.) and an index fund, and we want to allocate a fixed budget across these assets. We might want to minimize the portfolio volatility while achieving at least a 10% annual return. This is directly a constrained optimization. The data needed (asset expected returns and covariances) can be estimated from publicly available sources: e.g., one can use Yahoo Finance or Kaggle datasets for historical prices to compute average returns and the covariance matrix of returns ￼ ￼. Python’s pandas_datareader or yfinance APIs allow fetching daily price data for stocks like Moderna, Pfizer, J&J, etc., over a time period ￼, from which returns can be calculated.

In summary, portfolio optimization provides a statistical computing example of constrained optimization. It’s data-driven (uses historical market data), involves a quadratic objective and linear constraints (all differentiable), and is typically solved with Lagrange multiplier methods in closed-form or with QP solvers. It underpins many financial decision-support systems ￼.

## Mathematical Formulation (Objective & Constraints)

We will formulate the problem as: minimize portfolio risk (variance) subject to achieving a target return and fully investing our budget. Let $w = (w_1,\dots,w_m)$ be the weight of capital invested in each of $m$ assets (e.g. $w_1=0.25$ means 25% of funds in asset 1). We have:

- An expected return vector $\mu = (\mu_1,\dots,\mu_m)$ where $\mu_j = E[\text{return of asset }j]$ (mean return). These can be estimated from historical average returns.

- A covariance matrix $\Sigma \in \mathbb{R}^{m\times m}$ where $\Sigma_{ij} = \mathrm{Cov}(r_i, r_j)$ is the covariance between returns of asset $i$ and $j$. The portfolio variance is $w^T \Sigma w$.

- A desired target return level $R_{\text{target}}$ (e.g. 10% annually).



The constrained optimization problem is:

$$
\begin{aligned}
& \min_{w_1,\dots,w_m} && w^T \Sigma w \quad && \text{(minimize variance)} \\
& \text{subject to} && w^T \mu \ge R_{\text{target}}, \quad && \text{(expected return at least target)} \\
& && \sum_{j=1}^m w_j = 1, \quad && \text{all budget invested} \\
& && w_j \ge 0 \quad \forall j~. \quad && \text{(no short-selling constraint)}
\end{aligned}
$$

This is a convex quadratic program. The objective $f(w) = w^T \Sigma w$ is a convex quadratic function (assuming $\Sigma$ is positive semi-definite). The constraints are linear: the return constraint $w^T\mu \ge R_{\text{target}}$ (which can also be written as equality $=R_{\text{target}}$ for an efficient frontier point), the budget constraint $\sum_j w_j = 1$, and optional non-negativity $w_j\ge0$ to prohibit short positions. In practice, allowing negative $w_j$ means you can short-sell some assets; that makes the problem unconstrained in sign. We include $w_j\ge0$ here to reflect a common real-world restriction (long-only portfolio).

**Interpretation:** We are looking for portfolio weights that minimize risk given a minimum return requirement. Alternatively, one can invert this to maximize return for a given risk – both formulations are equivalent in tracing out the efficient frontier. For example, one could solve $\max_w w^T\mu$ subject to $w^T\Sigma w \le \sigma^2_{\max}$ and $\sum w_j=1, w_j\ge0$, which means “maximize return for a risk not exceeding $\sigma_{\max}$” ￼. Either way, the efficient frontier emerges by varying the target return or risk limit.

In matrix form, using $\mathbf{1}$ to denote the vector of all ones: the constraints are $w^T \mu \ge R_{\text{target}}$ and $w^T \mathbf{1} = 1$, $w \ge 0$. This formulation is known as the Markowitz model of portfolio selection. The feasible region (defined by linear constraints) is convex and bounded (a simplex intersected with a half-space of return $\ge R_{\text{target}}$), and the objective is convex, so a global optimum exists. If short-selling is disallowed ($w\ge0$), the problem remains convex but no closed-form solution exists; if short-selling is allowed, we have an equality-constrained QP that can be solved analytically via KKT.



**Qestion: Apply Lagrangian and KKT Conditions** to solve this optimization problem. Derive the dual problem and explain how it can be solved.



## Toy Data

**Q: Use `cvxpy` to solve this real data problem.** Define the annual return vector $\mu$ and the covariance matrix $\Sigma$. Report the optimal weights of the stocks.

In [1]:
import cvxpy as cp
import yfinance as yf

tickers = ['AAPL','MSFT','GOOG','AMZN','JPM','WMT','GE','XOM','PG','T']
start = '2020-01-01'
end = '2022-01-01'

# Fetch adjusted close prices
data = yf.download(tickers, start=start, end=end)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed


In [2]:
# Extract the close prices for all tickers
close_prices = data['Close']

# Optional: check the shape and preview
print(close_prices.shape)
print(close_prices.head())

(505, 10)
Ticker           AAPL       AMZN         GE       GOOG         JPM  \
Date                                                                 
2020-01-02  72.716064  94.900497  58.053150  68.046196  120.733543   
2020-01-03  72.009117  93.748497  58.247799  67.712280  119.140274   
2020-01-06  72.582901  95.143997  59.075039  69.381874  119.045540   
2020-01-07  72.241547  95.343002  58.637081  69.338585  117.021729   
2020-01-08  73.403633  94.598503  58.101814  69.884995  117.934601   

Ticker            MSFT          PG          T        WMT        XOM  
Date                                                                 
2020-01-02  153.323242  107.748268  19.607115  36.681835  55.137985  
2020-01-03  151.414108  107.023590  19.708025  36.358021  54.694691  
2020-01-06  151.805481  107.172005  19.728207  36.284004  55.114662  
2020-01-07  150.421402  106.508438  19.803892  35.947834  54.663582  
2020-01-08  152.817322  106.962463  19.864435  35.824478  53.839252  
